# Language detection: Spacy vs Pycld2

This notebook will compare the language detection between Spacy and Pycld2

In [1]:
import pandas as pd
from colorama import Fore

In [2]:
import spacy
from spacy_langdetect import LanguageDetector

In [3]:
import pycld2 as cld2

In [4]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 1

In [12]:
def load_data():
    languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", 
                 "en":"en_core_web_sm", "xx":"" }

    data_total = pd.DataFrame(columns=["Comentari","TipusIncidencia","Idioma"])

    for language in languages:
        model = languages[language]
        file = "comentaris_" + language + ".csv"
        if (debug >= 1):
            print ("language: ", language)
            print ("model: ", model)
            print ("file:", file)

        # Load data from file

        data_csv = pd.read_csv(pathdest + file)
        data_lang = data_csv[["Comentari","TipusIncidencia","Idioma"]]
        if (debug >= 2):
            display (data_lang.sample(5))
        if (debug >= 1):
            print ("rows:", data_lang.shape[0])
            print ("")

        data_total = pd.concat([data_total, data_lang])
        
    data_total = data_total[data_total["Comentari"].notnull()]
    data_total = data_total.reset_index()
    if (debug >= 1):
        print ("data_total: ", data_total.shape[0], " rows")

    if (debug >= 2):
        display (data_total.sample(5))
        
    return data_total

In [6]:
def detect_language_pycld2(comment):
    isReliable, textBytesFound, details = cld2.detect(comment.encode('utf-8', 'replace'), 
                                                    isPlainText = True, bestEffort = True, returnVectors=False)
    
    return details[0][1]

In [13]:
data = load_data()
if (debug >= 1):
    display(data)
    
data = detect_language_rows(data)
if (debug >= 1):
    display(data)

language:  ca
model:  ca_fasttext_wiki
file: comentaris_ca.csv
rows: 5999

language:  es
model:  es_core_news_sm
file: comentaris_es.csv
rows: 1345

language:  en
model:  en_core_web_sm
file: comentaris_en.csv
rows: 330

language:  xx
model:  
file: comentaris_xx.csv
rows: 185

data_total:  7855  rows


,index,Comentari,TipusIncidencia,Idioma
0,0,Els continguts d'aquesta assignatura ja els ha...,NaN,ca
1,1,L'actitud de la professora em resulta molt mot...,NaN,ca
2,2,Materials molt interessants i bones orientacio...,NaN,ca
3,3,Ha estat una exigència molt forta de treball: ...,NaN,ca
4,4,Molta satisfacció amb la transparència i l'ant...,NaN,ca
...,...,...,...,...
7850,180,Un exemple de bon professor,NaN,fr
7851,181,No sé quién es,No ha impartit classe a aquest grup,fr
7852,182,PROFE TOP,NaN,vi
7853,183,Sense paraules...,NaN,fr


In [14]:
#data["IdiomaPycld2"] = data.apply(lambda row: detect_language_pycld2(row["Comentari"]), axis=1)    
#data["IdiomaPycld2"] = detect_language_pycld2(data["Comentari"])

# Obtain IdiomaPycld2
def detect_language_rows(data):
    for x in range(data.shape[0]):
        comment = data.loc[x]["Comentari"]   
        idiomaPycld2 = detect_language_pycld2(comment)
        data.loc[x, "IdiomaPycld2"] = idiomaPycld2

    return data

,index,Comentari,TipusIncidencia,Idioma,IdiomaPycld2
0,0,Els continguts d'aquesta assignatura ja els ha...,NaN,ca,ca
1,1,L'actitud de la professora em resulta molt mot...,NaN,ca,pt
2,2,Materials molt interessants i bones orientacio...,NaN,ca,ca
3,3,Ha estat una exigència molt forta de treball: ...,NaN,ca,ca
4,4,Molta satisfacció amb la transparència i l'ant...,NaN,ca,ca
...,...,...,...,...,...
7850,180,Un exemple de bon professor,NaN,fr,en
7851,181,No sé quién es,No ha impartit classe a aquest grup,fr,es
7852,182,PROFE TOP,NaN,vi,en
7853,183,Sense paraules...,NaN,fr,ca


In [9]:
#   Distribution by Idioma        
data_group = data[["Idioma","Comentari"]].groupby(["Idioma"]).count()
if debug:
    print ("Idioma Spacy:")
    display (data_group.sort_values("Comentari", ascending=False).head(10))

Idioma Spacy:


,Comentari
Idioma,
ca,5991
es,1345
en,330
it,42
fr,42
pt,38
UNKNOWN,10
de,8
hu,6


In [10]:
#   Distribution by IdiomaPycld2        
data_group = data[["IdiomaPycld2","Comentari"]].groupby(["IdiomaPycld2"]).count()
if debug:
    print ("Idioma Pycld2:")
    display (data_group.sort_values("Comentari", ascending=False).head(10))

Idioma Pycld2:


,Comentari
IdiomaPycld2,
ca,5962
es,1343
en,426
fy,25
tk,14
un,13
hu,13
oc,8
pt,7


In [11]:
#   Comparativa idiomas        
data_group = data[["Idioma","IdiomaPycld2","Comentari"]].groupby(["Idioma","IdiomaPycld2"]).count().reset_index()
display (data_group.sort_values("Comentari", ascending=False).head(10))
data_pivot = data_group.pivot(index="Idioma", columns="IdiomaPycld2", values="Comentari")
if debug:
    display (data_pivot)

,Idioma,IdiomaPycld2,Comentari
3,ca,ca,5815
31,es,es,1297
24,en,en,239
5,ca,en,135
23,en,ca,61
53,it,ca,23
30,es,en,20
74,pt,es,19
41,fr,ca,16
29,es,ca,16


IdiomaPycld2,af,ca,crs,cs,cy,de,en,es,fi,fr,...,lb,ms,oc,pt,sk,tk,tn,un,xh,yo
Idioma,,,,,,,,,,,,,,,,,,,,,
UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
af,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ca,NaN,5815.0,NaN,NaN,NaN,1.0,135.0,11.0,NaN,1.0,...,1.0,1.0,4.0,6.0,1.0,NaN,NaN,2.0,NaN,1.0
cy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
de,NaN,3.0,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
en,NaN,61.0,NaN,NaN,NaN,NaN,239.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
es,NaN,16.0,NaN,NaN,NaN,NaN,20.0,1297.0,NaN,NaN,...,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
et,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fi,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
data_dif = data[data["Idioma"] != data["IdiomaPycld2"]]
display (data_dif)
print (data_dif.shape[0])

,index,Comentari,Idioma,idioma_pycld2,IdiomaPycld2
1,1,L'actitud de la professora em resulta molt mot...,ca,,pt
53,53,M'ha sembla correcta.,ca,,ie
54,54,Magnífica.,ca,,es
63,63,Falten exercicis per fer.,ca,,de
72,72,És un incompetent,ca,,hu
...,...,...,...,...,...
7850,172,No sé quién es,fr,,es
7851,173,PROFE TOP,vi,,en
7852,174,Les professores a vegades et diuen coses contr...,fr,,ca
7853,175,Sense paraules...,fr,,ca


500


In [99]:
data_dif = data[(data["Idioma"] == "ca") & (data["IdiomaPycld2"] != "ca") & (data["IdiomaPycld2"] != "es") & (data["IdiomaPycld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,Idioma,idioma_pycld2,IdiomaPycld2
1,1,L'actitud de la professora em resulta molt mot...,ca,,pt
53,53,M'ha sembla correcta.,ca,,ie
63,63,Falten exercicis per fer.,ca,,de
72,72,És un incompetent,ca,,hu
111,111,Per venir a vendre la empresa només...,ca,,fr
201,201,Teoria rapida i temps de practica a classe. Co...,ca,,un
250,250,És un professor correcte.,ca,,hu
365,365,"No puc evaluar-lo, no estava quan va fer classe.",ca,,pt
367,367,Sempre disposat a ajudar en el que necessitis.,ca,,pt
402,402,És una crack!!,ca,,hu


31  rows


In [102]:
data_dif = data[(data["Idioma"] == "es") & (data["IdiomaPycld2"] != "es") & (data["IdiomaPycld2"] != "ca") 
                & (data["IdiomaPycld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,Idioma,idioma_pycld2,IdiomaPycld2
6026,21,Materiales estupendos para disfrutar y aprender,es,,id
6168,163,No vimos a en clase a este docente,es,,gl
6256,251,No vimos clase con él,es,,gl
6257,252,No vimos clase con ella,es,,gl
6334,329,Excelente para la formación de Master,es,,gl
6345,340,mala organización de la asignatura,es,,sk
6410,405,Los testes de teoria son tediosos y dificiles.,es,,oc
6606,601,De dificil acceso y ausente durante el curso.,es,,gl
6770,765,No fica examen del teoria i fica coses que no ...,es,,pt
6818,813,Bona profe.,es,,fy


13  rows


In [105]:
data_dif = data[(data["Idioma"] == "en") & (data["IdiomaPycld2"] != "en") & (data["IdiomaPycld2"] != "ca")
                & (data["IdiomaPycld2"] != "es")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,Idioma,idioma_pycld2,IdiomaPycld2
7349,0,Gran profesor,en,,fy
7351,2,No l'hem tingut,en,,tk
7359,10,No l’hem tingut,en,,tk
7360,11,No l'hem tingut,en,,tk
7362,13,No l’hem tingut,en,,tk
7368,19,És mlt bon professor ;),en,,hu
7442,93,No l'hem tingut,en,,tk
7451,102,No l’hem tingut,en,,tk
7458,109,No l’hem tingut,en,,tk
7500,151,No l'hem tingut.,en,,tk


25  rows


In [110]:
data_dif = data[(data["Idioma"] != "ca") & (data["Idioma"] != "es") & (data["Idioma"] != "en")
                & (data["IdiomaPycld2"] != "ca") & (data["IdiomaPycld2"] != "es") & (data["IdiomaPycld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,Idioma,idioma_pycld2,IdiomaPycld2
7687,9,`............````.`......`.`````.................,so,,xh
7689,11,¯\_(?)_/¯,UNKNOWN,,un
7692,14,No em crida l'atenció,pt,,pt
7697,19,Excelente docente,pt,,cs
7698,20,Excelente docente.,pt,,cs
7707,29,Excelente docente,pt,,cs
7710,32,Excelente profesor.,ro,,fy
7717,39,Excelente profesora,ro,,fy
7718,40,No tuvimos clase,lt,,un
7719,41,Comença les classes a les 16.10h en compte de ...,fr,,fr


55  rows


In [17]:
# Registro con idioma diferente agrupados por tipo de incidencia

data_dif = data[data["Idioma"] != data["IdiomaPycld2"]]
data_incid = data_dif[data_dif["TipusIncidencia"].notnull()]
data_group = data_incid[["TipusIncidencia","Comentari"]].groupby(["TipusIncidencia"]).count()
data_group

,Comentari
TipusIncidencia,
Comentari de professor,2
Comentari excel·lent,1
Comentari ofensiu,2
Comentari problemàtic,5
Exclamacions o emoticones excessius,1
No ha impartit classe a aquest grup,58


Nivel general de coincidencia:
- Textos con igual idioma: 7.351 / 7855 = 93,58 %
- Textos con idioma diferente: 504 registros. Candidatos a recuperarse.

Revisión idioma catalán:
- Los textos detectados ca (spacy) - en (pycld2) són en realidad en catalán (141 casos)
- Los textos detectados ca (spacy) - es (pycld2) són en realidad en catalán (11 casos)
- Los textos detectados ca (spacy) - xx (pycld2) són en realidad en catalán (31 casos)
Conclusión: En general la detección de catalán de Spacy es correcta

Revisión idioma castellano:
- Los textos detectados es (spacy) - ca (pycld2) són en realidad en catalán (16 casos)
- Los textos detectados es (spacy) - en (pycld2) són en realidad en castellano (19 casos)
- Los textos detectados es (spacy) - xx (pycld2) són en realidad en castellano (13 casos)
Conclusión: En geneal la detección del castellano en Spacy es correcta, salvo para el idioma catalán, que se podria mejorar mediante pycld2 

Revisión idioma inglés:
- Los textos detectados en (spacy) - ca (pycld2) són en realidad en catalán (60 casos)
- Los textos detectados en (spacy) - es (pycld2) són en realidad en castellano (8 casos)
- Los textos detectados en (spacy) - xx (pycld2) són en realidad en catalán (25 casos)
Conclusión: Para el castellano y el catalán la detección de pycld2 es mejor y en el caso de otras lenguas se debería hacer una asignación directa al catalán.

Revisión otras lenguas (xx representa las lengua que no son ca, es, en):
- Los textos detectados xx (spacy) - ca (pycld2) són en realidad en catalán (68 casos)
- Los textos detectados xx (spacy) - es (pycld2) són en realidad en castellano (28 casos)
- Los textos detectados xx (spacy) - en (pycld2) són en realidad en catalán (30 casos)
- Los textos detectados xx (spacy) - xx (pycld2) són casi todos catalán y el resto castellano (55 casos)

Conclusión: Para el castellano y el catalán la detección de pycld2 es mejor y en el caso de otras lenguas se debería hacer una asignación directa al catalán.

Agrupación por tipos de incidencia:
- De los 505 registro recuperables 69 son incidencias.
- La mayoría de las incidencias (58 registros) son de tipo 'No ha impartido clase'.
- En todo el conjunto de datos hay 440 incidencias, 37 de tipo 'No ha impartido clase'.
